In [ ]:
import os
import zipfile
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# ========== 1. 解壓 processed_data.zip ==========
zip_path = '/content/processed_data.zip'
extract_path = '/content/data/processed/'
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"processed_data.zip 已解壓縮到 {extract_path}")

# ========== 2. 載入資料 ==========
X = np.load(os.path.join(extract_path, 'X_train.npy'))
Y = np.load(os.path.join(extract_path, 'Y_train.npy'))
print(f"成功讀取：X形狀 {X.shape}，Y形狀 {Y.shape}")

# ========== 3. Tensor 轉換與清除 NaN ==========
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
mask = ~torch.isnan(X_tensor).any(dim=(1, 2)) & ~torch.isnan(Y_tensor).any(dim=1)
X_tensor = X_tensor[mask]
Y_tensor = Y_tensor[mask]
print(f"清除 NaN 後資料形狀：X {X_tensor.shape}, Y {Y_tensor.shape}")

# ========== 4. 訓練集 / 測試集切分 ==========
total_len = len(X_tensor)
train_len = int(total_len * 0.8)
test_len = total_len - train_len
train_X, test_X = torch.utils.data.random_split(TensorDataset(X_tensor, Y_tensor), [train_len, test_len])
train_loader = DataLoader(train_X, batch_size=512, shuffle=True)
test_loader = DataLoader(test_X, batch_size=512, shuffle=False)
print(f"資料切分：訓練 {train_len} 筆，測試 {test_len} 筆")

# ========== 5. 定義 LSTM 模型 ==========
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

input_dim = X.shape[2]
output_dim = Y.shape[1]
model = LSTMModel(input_dim, hidden_dim=64, output_dim=output_dim).to('cuda' if torch.cuda.is_available() else 'cpu')
print("LSTM 模型建立完成")

# ========== 6. 訓練模型（雨日加權） ==========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.0003)
epochs = 50
loss_history = []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss_raw = criterion(pred, yb)
        weight = (yb[:, 0] > 0).float() * 1.0 + 1.0
        loss = (loss_raw.mean(dim=1) * weight).mean()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    loss_history.append(avg_loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Weighted Loss: {avg_loss:.6f}")

print("LSTM 模型訓練完成")

# ========== 7. 評估 ==========
model.eval()
with torch.no_grad():
    pred_all = []
    true_all = []
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred_all.append(model(xb).cpu().numpy())
        true_all.append(yb.numpy())

pred = np.concatenate(pred_all, axis=0)
true = np.concatenate(true_all, axis=0)

mae = mean_absolute_error(true, pred)
rmse = np.sqrt(mean_squared_error(true, pred))
r2 = r2_score(true, pred)

print(f"\n測試集評估結果：MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

# ========== 8. 儲存與繪圖 ==========
os.makedirs('/content/models/', exist_ok=True)
torch.save(model.state_dict(), '/content/models/lstm_model.pth')
np.save('/content/models/loss_history_lstm.npy', np.array(loss_history))

# 8-1. Loss 曲線
plt.figure(figsize=(8, 5))
plt.plot(loss_history, label='Loss')
plt.title('LSTM Training Loss Curve (Rain-weighted)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.savefig('/content/models/loss_curve_lstm.png')
plt.show()

# 8-2. 預測 vs 實際
plt.figure(figsize=(12, 6))
plt.plot(true[:200, 0], label='Actual', linestyle='-')
plt.plot(pred[:200, 0], label='Predicted', linestyle='--')
plt.title('LSTM: Actual vs Predicted Rainfall (Test Set, First 200 Samples)')
plt.xlabel('Time Step')
plt.ylabel('Precp')
plt.grid()
plt.legend()
plt.savefig('/content/models/lstm_prediction_curve.png')
plt.show()

# 8-3. 誤差分布圖
error = true[:, 0] - pred[:, 0]
plt.figure(figsize=(8, 5))
plt.hist(error, bins=50, color='lightblue', edgecolor='black')
plt.title('LSTM Prediction Error Distribution (Test Set)')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.grid()
plt.savefig('/content/models/lstm_error_hist.png')
plt.show()
